In [4]:
import os
import cv2
import numpy as np
from albumentations import (
    Compose, RandomBrightnessContrast, MotionBlur, GaussNoise, RandomRain,
    RandomFog, RandomShadow, RandomSunFlare, Blur
)
from albumentations.augmentations.transforms import ColorJitter
from albumentations.core.composition import OneOf
from tqdm import tqdm

In [5]:
# Input/Output paths
input_dir = "/content/drive/MyDrive/plusDataset_TSNeu"
output_dir = "plusDataset_TSNeu_augm"
os.makedirs(output_dir, exist_ok=True)

In [6]:
# Augmentation pipeline
augmentations = Compose([
    OneOf([
        RandomBrightnessContrast(brightness_limit=0.4, contrast_limit=0.4, p=1),
        ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1, p=1),
    ], p=0.9),

    OneOf([
        MotionBlur(blur_limit=7, p=1),
        Blur(blur_limit=5, p=1),
    ], p=0.6),

    OneOf([
        GaussNoise(var_limit=(10.0, 50.0), mean=0, p=1),
        RandomRain(p=1, blur_value=3, brightness_coefficient=0.9),
    ], p=0.7),

    OneOf([
        RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.5, alpha_coef=0.1, p=1),
        RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=5, p=1),
    ], p=0.6),

    RandomSunFlare(flare_roi=(0, 0, 1, 1), angle_lower=0.3, num_flare_circles_lower=3, src_radius=50, p=0.2),
])

def augment_images(input_folder, output_folder, num_augments=30):
    image_list = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    for img_name in tqdm(image_list, desc="Augmenting images"):
        img_path = os.path.join(input_folder, img_name)
        image = cv2.imread(img_path)

        for i in range(num_augments):
            augmented = augmentations(image=image)['image']
            base_name = os.path.splitext(img_name)[0]
            new_filename = f"{base_name}_aug_{i}.jpg"
            cv2.imwrite(os.path.join(output_folder, new_filename), augmented)

<ipython-input-6-58d48d5e8460>:14: UserWarning: Argument(s) 'var_limit, mean' are not valid for transform GaussNoise
  GaussNoise(var_limit=(10.0, 50.0), mean=0, p=1),
<ipython-input-6-58d48d5e8460>:19: UserWarning: Argument(s) 'fog_coef_lower, fog_coef_upper' are not valid for transform RandomFog
  RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.5, alpha_coef=0.1, p=1),
<ipython-input-6-58d48d5e8460>:20: UserWarning: Argument(s) 'num_shadows_lower, num_shadows_upper' are not valid for transform RandomShadow
  RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=5, p=1),
<ipython-input-6-58d48d5e8460>:23: UserWarning: Argument(s) 'angle_lower, num_flare_circles_lower' are not valid for transform RandomSunFlare
  RandomSunFlare(flare_roi=(0, 0, 1, 1), angle_lower=0.3, num_flare_circles_lower=3, src_radius=50, p=0.2),


In [ ]:
# Run augmentation
augment_images(input_dir, output_dir, num_augments=30)

Augmenting images: 100%|██████████| 90/90 [34:59<00:00, 23.32s/it]


In [2]:
import os

def count_images(folder_path):
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(image_extensions)]
    return len(image_files)

# Example usage:
folder = "//content/drive/MyDrive/plusDataset_TSNeu_augm"  # Change this to your folder path
num_images = count_images(folder)
print(f"Total images in '{folder}': {num_images}")


Total images in '//content/drive/MyDrive/plusDataset_TSNeu_augm': 2700


In [9]:
import shutil

image_list = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
# Copy original images to the augmented folder
for img_name in image_list:
    shutil.copy(os.path.join(input_dir, img_name), os.path.join("/content/drive/MyDrive/plusDataset_TSNeu_augm", img_name))


In [10]:
# verify lenth of output_dir now
folder = "//content/drive/MyDrive/plusDataset_TSNeu_augm"  # Change this to your folder path
num_images = count_images(folder)
print(f"Total images in '{folder}': {num_images}")

Total images in '//content/drive/MyDrive/plusDataset_TSNeu_augm': 2790


*  **Upload to roboflow**

In [11]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [12]:
from roboflow import Roboflow
rf = Roboflow(api_key="Tz5FdPrgSydq1RLk8NSt")


In [16]:
# 🧭 Step 2: Select your workspace and project
project = rf.workspace("zahra-abouhane-b7ahw").project("neurodrivearticle_dataaug") # Replace with your info
dataset = project.version(1)  # Adjust version if needed

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
project = rf.workspace("zahra abouhane").project("neurodrivearticle_dataaug")
dataset = project.version(1)

# Example: Upload one image (you can loop over a folder)
image_path = "/content/drive/MyDrive/your_images/image_1.jpg"
dataset.upload(image_path)


In [ ]:
#full code for importing to roboflow with loop
!pip install roboflow
from roboflow import Roboflow
import os

# 🔑 Step 1: Authenticate with your Roboflow API key
rf = Roboflow(api_key="YOUR_API_KEY")  # ← Replace with your real key

# 🧭 Step 2: Select your workspace and project
project = rf.workspace("your-workspace").project("your-project")  # Replace with your info
dataset = project.version(1)  # Adjust version if needed

# 📁 Step 3: Define the folder containing your images
image_folder = "/content/drive/MyDrive/your_folder"  # Mount your Drive first if not done

# 📤 Step 4: Upload all images in a loop
def upload_all_images(folder):
    image_extensions = ('.jpg', '.jpeg', '.png')
    uploaded = 0
    for filename in os.listdir(folder):
        if filename.lower().endswith(image_extensions):
            image_path = os.path.join(folder, filename)
            try:
                dataset.upload(image_path)
                uploaded += 1
                print(f"[{uploaded}] Uploaded: {filename}")
            except Exception as e:
                print(f"❌ Failed: {filename} | Error: {e}")

upload_all_images(image_folder)


In [17]:
# Install
#!pip install roboflow

# Authenticate
!roboflow login

# Import
!roboflow import -w zahra-abouhane-b7ahw -p neurodrivearticle_dataaug /content/drive/MyDrive/plusDataset_TSNeu_augm


visit https://app.roboflow.com/auth-cli to get your authentication token.
Paste the authentication token here: 
loading Roboflow workspace...
100% 2790/2790 [00:00<00:00, 2334352.32it/s]
loading Roboflow project...
Uploading to existing project zahra-abouhane-b7ahw/neurodrivearticle_dataaug
[UPLOADED] /content/drive/MyDrive/plusDataset_TSNeu_augm/WhatsApp Image 2025-05-07 at 16.12.47.jpeg (nnftdH1xAkEGbhltFUNl) [2.1s]
[UPLOADED] /content/drive/MyDrive/plusDataset_TSNeu_augm/WhatsApp Image 2025-05-07 at 16.12.48.jpeg (jwwNqYW7KvJRspD0JrRp) [2.2s]
[UPLOADED] /content/drive/MyDrive/plusDataset_TSNeu_augm/WhatsApp Image 2025-05-07 at 16.12.49.jpeg (jotqzBhIo9caO8ROriZc) [2.2s]
[UPLOADED] /content/drive/MyDrive/plusDataset_TSNeu_augm/WhatsApp Image 2025-05-07 at 16.12.46.jpeg (fzRByi44OQ6t6AibOL9v) [2.3s]
[UPLOADED] /content/drive/MyDrive/plusDataset_TSNeu_augm/WhatsApp Image 2025-05-07 at 16.12.00_aug_1.jpg (d7umtkjYjuherqRfIb3u) [2.6s]
[UPLOADED] /content/drive/MyDrive/plusDataset_TSNeu_a